# SMAP data processing demonstration
The below script provides examples on how to:
1. ### Read in SMAP data and navigate metadata
2. ### Create a map with SMAP data
3. ### Plot a time-series at a location on Earth

The complementary <b><i>1.0 Access SMAP data</i></b> script provided demonstrates where and how to obtain SMAP data from the [NSIDC data portal](https://n5eil01u.ecs.nsidc.org/SMAP/) via [WGET](https://www.gnu.org/software/wget/) and [OPENDAP](https://www.opendap.org).

#### Import packages

In [ ]:
import datetime as dt
import glob
import h5py
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import os
import pandas as pd
import xarray as xr

## 1. Read SMAP data and navigate metadata
* First we navigate to the folder with the data we want to use for this quick demo. 
* Next lets make a list of the files within the folder and print one out.

In [ ]:
this_dir = os.getcwd()
L3_SM_P_dir = os.path.join(this_dir, 'data/L3_SM_P/')

flist = glob.glob(os.path.join(L3_SM_P_dir, '*.h5'))
           
filename = flist[0]; 
print("File we are using: " + filename + '\n')

*  Now lets use h5py.File() to open the file
*  Then we can look at the folders within the file to access the data we want

In [ ]:
f = h5py.File(filename, 'r')
print('By using the command h5py.File() a filehandle is returned:')
print(f); print('\n')

print("Now lets look at the groups within the file to access:")
i=0;
for key in f.keys():
    print(str(i)+ '\t'+key)
    i+=1
group_id=list(f.keys())[1];# < Lets focus on the AM overpass for this example
print('\n')
i=0
print("Now lets look at the variables within the filegroup **Soil_Moisture_Retrieval_Data_AM** to access the actual data:")
for var in list(f[group_id].keys()):
    print(str(i)+'\t'+var)
    i+=1

Ok now that we know the variables within the Soil_Moisture_Retrieval_Data_AM lets take a grab the data we want to make a plot of soil moisture.
* First lets take a look at the shape
    - we will need this later when opening the lat, lon datasets

In [ ]:
print('the data has a shape of: '+str(f[group_id][list(f[group_id].keys())[0]].shape))

* Now fill in the below keys from above to get the variables we want to plot.

In [ ]:
var_id = list(f[group_id].keys())[16] # soil_moisture
sm_data = f[group_id][var_id][:,:]
sm_ds = f[group_id][var_id]
print('data are returned as '+str(type(sm_data)) + ' something easy to work with in python.')

In [ ]:
ret_flag_L3_P = f[group_id]['retrieval_qual_flag'][:,:]
print(np.unique(ret_flag_L3_P))
print(type(ret_flag_L3_P))

## 2. Create a map with SMAP data
* Lets plot the data to get a sense of what we are working with

In [ ]:
plt.imshow(sm_data)
cbar = plt.colorbar(orientation = 'horizontal')
cbar.set_label('$cm^3 cm^{-3}$')

Ok we have to ignore the null values to get a better idea of the range in values of the dataset.

You can find these null values in the meta data of the supplemental documents.

We know for soil moisture and most smap datasets it to be -9999.
This is located under the attributes for the 'soil_moisture' variable

In [ ]:
print(f[group_id][var_id].attrs['_FillValue'])

In [ ]:
sm_data[sm_data==f[group_id][var_id].attrs['_FillValue']]=np.nan;
plt.imshow(sm_data,vmin=0.,vmax=0.55, cmap = 'terrain_r');
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label('$cm^3 cm^{-3}$')

How about we spruce this up with some coastlines, and geolocate the data using the lat, lon data?

Below we load the EASE2 grid lon and lat datasets.  These can be found on the NSIDC website.

In [ ]:
# Read binary files and reshape to correct size
lats = np.fromfile('EASE2_M36km.lats.964x406x1.double', 
                      dtype=np.float64).reshape((406,964))#< reshape to dimensions above
lons = np.fromfile('EASE2_M36km.lons.964x406x1.double', 
                      dtype=np.float64).reshape((406,964))

Now lets use basemap to plot the dataset

There are many online tutorials for basemap, to focus in on a region, and it provides lots of ways to display the data.

In [ ]:
fig = plt.figure(figsize=(10,6))
m = Basemap(resolution='l',projection='robin',lat_ts=40,lat_0=lats.mean(),lon_0=lons.mean())
xi, yi = m(lons, lats)
cs = m.pcolor(xi,yi,sm_data, vmin = 0., vmax = 0.55, cmap = 'terrain_r')
m.drawcoastlines()
cbar = m.colorbar(cs, location='bottom', pad="5%")
cbar.set_label('$cm^3 cm^{-3}$')

Awesome! We now can make a global plot, lets see if we can make this a bit more streamlined to process more data and create a time series.

## 3. Plot a time-series at a location on Earth
* Lets stick with the L3 SM P dataset because the data isnt that big and its what I could put on your flashdrives for the demo.

Lets start by navigating back to the L3_SM_P directory & creating a filelist and a make a 3-d array to work wiht

In [ ]:
for fName in flist:
    print(fName)

* Lets make a function to load some data.



In [ ]:
def read_SML3P(filepath):
    ''' This function extracts lat, lon and soil moisture from SMAP L3 P HDF5 file.
    
    Parameters
    ----------
    filepath : str
        File path of a SMAP L3 HDF5 file
    Returns
    -------
    soil_moisture_am: numpy.array
    '''    
    with h5py.File(filepath, 'r') as f:
        # Extract data info
        group_id_am = 'Soil_Moisture_Retrieval_Data_AM'
        var_id_am = 'soil_moisture'
        flag_id_am = 'retrieval_qual_flag'
        soil_moisture_am = f[group_id_am][var_id_am][:,:]
        flag_am = f[group_id_am][flag_id_am][:,:]
        soil_moisture_am[soil_moisture_am==-9999.0]=np.nan;
        soil_moisture_am[(flag_am>>0)&1==1]=np.nan
        filename = os.path.basename(filepath)
        yyyymmdd= filename.split('_')[4]
        yyyy = int(yyyymmdd[0:4]);        mm = int(yyyymmdd[4:6]);        dd = int(yyyymmdd[6:8])
        date=dt.datetime(yyyy,mm,dd)
    return soil_moisture_am,date

* Lets check if it works:

In [ ]:
sm_test,date = read_SML3P(flist[0])
plt.imshow(sm_test)
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label('$cm^3 cm^{-3}$')
plt.title(date)

* Now that we have that working lets add everything to a 3-d array, remember time is the 3rd dimension
* For larger datasets (i.e. 3km data or even 9km data you need to get creative to do on your local machine)

In [ ]:
sm_data_3d = np.empty([sm_data.shape[0],sm_data.shape[1],len(flist)])
times = []
print('sm_data_3d has dimensions '+str(sm_data_3d.shape))
i=0
for fName in flist:
    sm_data_3d[:,:,i],time_i = read_SML3P(fName)
    times.append(time_i)
    i+=1

In [ ]:
sm_mean = np.nanmean(sm_data_3d,2)
sm_mean.shape
plt.imshow(sm_mean,vmin=0.,vmax=0.55,cmap='terrain_r')
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label('$cm^3 cm^{-3}$')

In [ ]:
N_lat = 37.5; 
S_lat = 33
W_lon = -113.5
E_lon = -110.0

subset = (lats<N_lat)&(lats>S_lat)&(lons>W_lon)&(lons<E_lon);
sm_time = np.empty([len(flist)]);
for i in np.arange(0,sm_data_3d.shape[2]):
    sm_2d = sm_data_3d[:,:,i]
    sm_time[i] = np.nanmean(sm_2d[subset]); 

# Lets create a pandas series to plot the data
smData = {'sm':sm_time}
sm = pd.DataFrame(smData)
sm['time']=times
sm=sm.set_index('time')

fig, ax1 = plt.subplots()
ax1.plot(sm.index, sm, 'b*')       
ax1.set_ylim([0,0.25])
fig.autofmt_xdate()
ax1.set_ylabel('$cm^3 cm^{-3}$')
plt.title('Soil Moisture')